# Nearest Neighbors of a  residue
### Author:
Syamanthaka Balakrishnan(SMBK)
### Created: May 2017
### Last modified: May 2017


## Aim:
To see which residues get affected if a given residue is mutated in a molecule. Hypothesis is that a position which could potentially affect a large group of primary and secondary positions, may tent to unstabilize the variant which is being mutated. Alternately, it may affect the performance of the variant.

## Description:
This is a very simple notebook that gives the nearest neighbors of a given residue of a molecule upto a radius of n Angstrom. Currently 2 levels of neighbors are extracted. The position which affects the maximum number of level 1 and level 2 neighbors are identified. It is being attempted to verify if these could potentially be detremental to the overall stability or performance of the variant and hence may be avoided from being changed. 
Contact @VASP for inspiration on the concept used here.

## User inputs:
* pdb file of the molecule of interest
* One or more residues which need to be studied
* Distance in Angstrom which is the radius of study

## Output format:
* A tabular format with column of the residue that was searched and it's neighbors. The neighbor list is in the format of 3 letter code and position, separated by a '-'. Eg. LEU-3 means Leusine in position 3

## Steps:
* Enter the 3 inputs mentioned above, in the right cells.
* Run all cells
* Use output table as you deem fit.
* Suggested extra step : credit author when using this notebook


### Import libraries

In [ ]:
## Necessary imports
from Bio.PDB import *
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Parse the structure
User input required here. Enter pdb file of interest

In [ ]:
## Create parser object
parser = PDBParser()
## Load the structure
# The first word in quotes is a nickname you give
# The second is the pdb file to be used. It needs to be uploaded to your notebook directory before
structure = parser.get_structure('Amy', '1bli.pdb') 
## Selecting the first model, chain A (for most cases)
chain = structure[0]['A'] #A instead of ' '


### The required functions

In [ ]:
## Get the first level neighbors
def get_neighbors(residue_pos, distance):
    try:
        center_residues = chain[residue_pos]
    except KeyError:
        return("na", 0,[])
    center_atoms = Selection.unfold_entities(center_residues, 'A')

    atom_list = [atom for atom in structure.get_atoms() if atom.name == 'CA']
    ns = NeighborSearch(atom_list) 
    
    nearby_residues = {res for center_atom in center_atoms
                   for res in ns.search(center_atom.coord, 4, 'R')}
   
    
    neighbor_list = [res.resname + "-" + str(res.id[1]) for res in nearby_residues if res.id[1] != residue_pos]
    neighbor_ids = [res.id[1] for res in nearby_residues]
    neighbor_ids.remove(residue_pos)
    neighbor_str = ','.join(map(str, neighbor_list))
    #print(neighbor_str) 
    return(neighbor_list, len(neighbor_list), neighbor_ids)


## Look up second level neighbors
def find_secondary(prim_list):
    secondary_lst = []
  
    for i in prim_list:
        try:
            the_neighs = df_primary.loc[df_primary['Search_Residue'] == i, 'Immediate_Neighbors'].item()
            secondary_lst.extend(the_neighs)
        except:
            return(["na"])
    secondary = list(set(secondary_lst))
    return(secondary)
    

### The results! 
User input required here - residues to be searched and distance

In [ ]:
## Use the first line if you only want to check for specific positions. Comment out the second line in that case
#all_residues = [3,5] # With the square brackets enter the position(s) of interest separated by ','
all_residues = [res.id[1] for res in Selection.unfold_entities(structure, 'R')]
radius = 4 # Enter distance in Angrstrom - a single number


imm_neighbors = []
imm_neighbor_count = []
imm_neighbor_list = []

for res in all_residues:
    neighbor_temp = get_neighbors(res, radius)
    if neighbor_temp == "na":
        next
    neighbor_res = neighbor_temp[0]
    neighbor_len = neighbor_temp[1]
    
    imm_neighbors.append(neighbor_res)
    imm_neighbor_count.append(neighbor_len)
    imm_neighbor_list.append(neighbor_temp[2])
    
df = pd.DataFrame()
df['Search_Residue'] = all_residues
df['Immediate_Neighbors'] = imm_neighbors
df['Imm_neighbor_count'] = imm_neighbor_count
df['Primary_list'] = imm_neighbor_list

df_primary = df[df.Imm_neighbor_count != 0] # Filter out ones which have neighbors

## Lookup the secondary neighbors from created data frame 
df_primary['Secondary_Neighbors'] = df_primary.apply(lambda row: find_secondary(row['Primary_list']), axis=1)
df_primary['Secondary_Neighbors_count'] = df_primary.apply(lambda row: len(row['Secondary_Neighbors']), axis=1)

del df_primary['Primary_list'] #Removing unwanted columns
df_primary

### Report

In [ ]:
# Count of max number of immediate neighbors affected
max_imm_neigh = df_primary['Imm_neighbor_count'].max()

imm_affected = df_primary[df_primary['Imm_neighbor_count'] == max_imm_neigh]

print(str(max_imm_neigh) + " primary positions will be affected by changes to position(s) below")
imm_affected


In [ ]:
# Count of max number of secondary neighbors affected
max_sec_neigh = df_primary['Secondary_Neighbors_count'].max()

sec_affected = df_primary[df_primary['Secondary_Neighbors_count'] == max_sec_neigh]
print(str(max_sec_neigh) + " secondary positions will be affected by changes to position(s) below")
sec_affected

In [ ]:
## Save to excel
df_primary.to_excel('Nearest_neighbors.xlsx', sheet_name = 'Report')
## This file would be available in your notebook directory on jupyter. 